In [39]:
import os, sys, numpy, torch, matplotlib.pyplot, matplotlib.cm, ipycanvas, matplotlib.patches

sys.path += [os.path.abspath(os.path.join('..')), os.path.abspath(os.path.join('../..'))]  # Allow repository modules to be imported

from functools import partial
from utils.optimization import initialize
from utils.plots import plot_train_loss, plot_samples_and_neurons, plot_weights_norms, draw_figure_into_canvas, save_figure
from utils.models import EpsilonNetFullyConnectedNeuralNetwork, FullyConnectedNeuralNetwork
from settings.sphere_2d_epsilon_net import get_dataloader, sphere_2d_epsilon_net
from experiment import model_summary, plot_neurons_inner_product_to_weights_products, execute_experiment

experiment = {
    'dataset': 'sphere_2d_epsilon_net',
    'epsilon': 0.6,
    'net_epsilon': .6,
    'seed': 2,
    'input_dimension': 2, 
    'sample_size': 100,  
    'batch_size': 100, 
    'within_cluster_variance': 0.0,
    'normalize_inputs':True,
    'epochs': 100000,
    'learning_rate': .1,
    'bias': False,
    'name_parameters': ['seed'],
    'balance_classes': True,
    'initialization_scale': 1.,
    'initial_hidden_units': 100,
    #'initial_weights': [
    #    [[0., 1e-5], [0., -1e-5], [-(.25**.5)*1e-5, -(7.5**.5)*1e-5], [(.5**.5)*1e-5, -(.5**.5)*1e-5], [-(.15**.5)*1e-5, -(.85**.5)*1e-5]]
    #],
    #'output_layer_initial_weights': [-1e-5, 1e-5, 1e-5, 1e-5, 1e-5]
}
rotation_matrix = numpy.identity(experiment['input_dimension'])
device, generator = initialize(experiment['seed'])
train_data = get_dataloader(**experiment, rotation_matrix=rotation_matrix, generator=generator)
test_data = get_dataloader(**experiment, rotation_matrix=rotation_matrix, generator=generator)


#X =[[2**-.5, 2**-.5], [2**-.5, -2**-.5],] + [[1, 0],] * 2
##X =[[1., 1.], [1., -1.],] + [[1., 0.],] * 2
#y = [0, 0, ] + [1, ] * 2
#
#with torch.no_grad():
#    tensor_X = torch.Tensor(X)
#    tensor_y = torch.Tensor(y)
#
#dataset = torch.utils.data.TensorDataset(tensor_X, tensor_y)
#train_data = test_data = torch.utils.data.DataLoader(dataset, len(X), shuffle=True)

fig, ax = matplotlib.pyplot.subplots(figsize=(10, 10))
inputs = []; labels = []
for batch_inputs, batch_labels in train_data: inputs.append(batch_inputs); labels.append(batch_labels)
inputs, labels = torch.concatenate(inputs), torch.concatenate(labels)
inputs_ = numpy.matmul(inputs.detach().cpu().numpy(), rotation_matrix.transpose())
ax.hlines(0, -inputs_.max() * 1.1, inputs_.max() * 1.1, color='k')
ax.vlines(0, -inputs_.max() * 1.1, inputs_.max() * 1.1, color='k')
ax.scatter(inputs_[:, 0], inputs_[:, 1], c=labels, cmap=matplotlib.cm.get_cmap('RdYlBu'))
nodes = sphere_2d_epsilon_net(**experiment)
#ax.add_patch(matplotlib.patches.Circle([0, 0], 1, color='k', alpha=.1, fill=False))
#for node in nodes:
#    ax.add_patch(matplotlib.patches.Circle(node, experiment['epsilon'], color='k', alpha=.05))

/tmp/ipykernel_24497/1253637130.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  ax.scatter(inputs_[:, 0], inputs_[:, 1], c=labels, cmap=matplotlib.cm.get_cmap('RdYlBu'))


In [40]:
matplotlib.pyplot.ioff()
figure, ((input_domain_ax, loss_ax), (parameters_norms_ax, inner_product_to_weights_product_ax)) = matplotlib.pyplot.subplots(2, 2, figsize=(16, 16))

canvas = ipycanvas.Canvas()
canvas.width, canvas.height = 1200, 1200
canvas.font = '30px arial'
canvas.fill_text('Results will appear as processed', canvas.width / 4, canvas.height / 3)

plot_samples_and_neurons = partial(plot_samples_and_neurons, ax=input_domain_ax, rotation_matrix=rotation_matrix, dataloader=train_data, label_neurons=True)
plot_train_loss = partial(plot_train_loss, ax=loss_ax)
plot_weights_norms = partial(plot_weights_norms, ax=parameters_norms_ax)
plot_neurons_inner_product_to_weights_products = partial(plot_neurons_inner_product_to_weights_products, ax=inner_product_to_weights_product_ax, aggregate_neurons=True)
draw_figure_into_canvas = partial(draw_figure_into_canvas, figure=figure, canvas=canvas)
save_figure = partial(save_figure, figure=figure, parameters=experiment, **experiment)

canvas

Canvas(height=1200, width=1200)

In [41]:
model = EpsilonNetFullyConnectedNeuralNetwork(**experiment, repeat_nodes=False)
plot_samples_and_neurons(model=model)
draw_figure_into_canvas()

In [35]:
nodes, model(torch.Tensor(nodes))

(array([[ 0.00000000e+00,  1.00000000e+00],
        [ 3.82683432e-01,  9.23879533e-01],
        [ 7.07106781e-01,  7.07106781e-01],
        [ 9.23879533e-01,  3.82683432e-01],
        [ 1.00000000e+00,  6.12323400e-17],
        [ 9.23879533e-01, -3.82683432e-01],
        [ 7.07106781e-01, -7.07106781e-01],
        [ 3.82683432e-01, -9.23879533e-01],
        [ 1.22464680e-16, -1.00000000e+00],
        [-3.82683432e-01, -9.23879533e-01],
        [-7.07106781e-01, -7.07106781e-01],
        [-9.23879533e-01, -3.82683432e-01],
        [-1.00000000e+00, -1.83697020e-16],
        [-9.23879533e-01,  3.82683432e-01],
        [-7.07106781e-01,  7.07106781e-01],
        [-3.82683432e-01,  9.23879533e-01]]),
 tensor([ 1.9891e-11, -1.9891e-11,  1.9891e-11, -1.9891e-11,  1.9891e-11,
         -1.9891e-11,  1.9891e-11, -1.9891e-11,  1.9891e-11, -1.9891e-11,
          1.9891e-11, -1.9891e-11,  1.9891e-11, -1.9891e-11,  1.9891e-11,
         -1.9891e-11], grad_fn=<SqueezeBackward0>))

In [4]:
experiment, model, device, generator = execute_experiment(
    **experiment,
    train_data=train_data, 
    test_data=test_data, 
    model_class=EpsilonNetFullyConnectedNeuralNetwork, 
    #model_class=FullyConnectedNeuralNetwork,
    saving_epochs_interval=experiment['epochs'],
    callbacks_epochs=list(range(0, experiment['epochs'], 10000)),
    callbacks=[model_summary, plot_samples_and_neurons, 
               plot_train_loss, plot_weights_norms, 
               plot_neurons_inner_product_to_weights_products, draw_figure_into_canvas, save_figure],
    overwrite=True,
    repeat_nodes=False
)
model(inputs)
outputs = model.activations[-1] @ torch.diag(model.output_layer.weight.flatten())#.sum(dim=1).shape
gamma = (torch.diag((labels * 2. - 1.).flatten()) @ outputs)
gamma_bar = gamma / gamma.sum(dim=1).min()

KeyboardInterrupt: 